In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# HugeCTR demo on Movie lens data

## Overview

HugeCTR is a recommender specific framework which is capable of distributed training across multiple GPUs and nodes for Click-Through-Rate (CTR) estimation. It is a component of NVIDIA [Merlin](https://developer.nvidia.com/nvidia-merlin#getstarted), which is a framework accelerating the entire pipeline from data ingestion and training to deploying GPU-accelerated recommender systems.

### Learning objectives

This notebook demonstrates the steps for training a deep learning recommender model (DLRM) on the movie lens 20M [dataset](https://grouplens.org/datasets/movielens/20m/). We will walk you through the process of data preprocessing, train a DLRM model with HugeCTR, then using the movie embedding to answer item similarity queries.

## Content
1. [Pre-requisite](#1)
1. [Data download and preprocessing](#2)
1. [HugeCTR DLRM training](#3)
1. [Answer item similarity with DLRM embedding](#4)


<a id="1"></a>
## 1. Pre-requisite


### 1.1 Docker containers
Please make sure that you have started the notebook inside the running NGC docker container: `nvcr.io/nvidia/hugectr:v3.0`. The binary executables have been installed to the system path `/usr/local/hugectr/bin/`. Besides, this system path is added to the environment variable `PATH`, which means that you can use the HugeCTR binary executables within the docker container environment.

### 1.2 Hardware
This notebook requires a Pascal, Volta, Turing, Ampere or newer GPUs, such as P100, V100, T4 or A100. 

In [2]:
!nvidia-smi

Thu Feb 11 00:00:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    44W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

<a id="2"></a>
## 2. Data download and preprocessing

We first install a few extra utilities for data preprocessing.

In [3]:
print("Downloading and installing 'tqdm' package.")
!pip3 -q install torch tqdm

print("Downloading and installing 'unzip' command")
!conda install -y -q -c conda-forge unzip

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - unzip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    unzip-6.0                  |       h7f98852_2         143 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         143 KB

The following NEW packages will be INSTALLED:

  unzip              conda-forge/linux-64::unzip-6.0-h7f98852_2


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Next, we download and unzip the movie lens 20M [dataset](https://grouplens.org/datasets/movielens/20m/).

In [4]:
print("Downloading and extracting 'Movie Lens 20M' dataset.")
!wget -nc http://files.grouplens.org/datasets/movielens/ml-20m.zip -P data -q --show-progress
!unzip -n data/ml-20m.zip -d data
!ls ./data

ml-20m.zip          100%[===================>] 189.50M  17.0MB/s    in 21s     
Archive:  data/ml-20m.zip
   creating: data/ml-20m/
  inflating: data/ml-20m/genome-scores.csv  
  inflating: data/ml-20m/genome-tags.csv  
  inflating: data/ml-20m/links.csv   
  inflating: data/ml-20m/movies.csv  
  inflating: data/ml-20m/ratings.csv  
  inflating: data/ml-20m/README.txt  
  inflating: data/ml-20m/tags.csv    
ml-20m	ml-20m.zip


### Movie lens data preprocessing

In [5]:
import pandas as pd
import torch
import tqdm

In [6]:
MIN_RATINGS = 20
USER_COLUMN = 'userId'
ITEM_COLUMN = 'movieId'

Next, we read the data into a Pandas dataframe, and encode userID and itemID with integers.

In [7]:
df = pd.read_csv('./data/ml-20m/ratings.csv')
print("Filtering out users with less than {} ratings".format(MIN_RATINGS))
grouped = df.groupby(USER_COLUMN)
df = grouped.filter(lambda x: len(x) >= MIN_RATINGS)

print("Mapping original user and item IDs to new sequential IDs")
df[USER_COLUMN], unique_users = pd.factorize(df[USER_COLUMN])
df[ITEM_COLUMN], unique_items = pd.factorize(df[ITEM_COLUMN])

nb_users = len(unique_users)
nb_items = len(unique_items)

print("Number of users: %d\nNumber of items: %d"%(len(unique_users), len(unique_items)))

# Save the mapping to do the inference later on
import pickle
with open('./mappings.pickle', 'wb') as handle:
    pickle.dump({"users": unique_users, "items": unique_items}, handle, protocol=pickle.HIGHEST_PROTOCOL)

Filtering out users with less than 20 ratings
Mapping original user and item IDs to new sequential IDs
Number of users: 138493
Number of items: 26744


Next, we split the data into a train and test set, the last movie each user has recently seen will be used for the test set.

In [8]:
# Need to sort before popping to get the last item
df.sort_values(by='timestamp', inplace=True)
    
# clean up data
del df['rating'], df['timestamp']
df = df.drop_duplicates() # assuming it keeps order

# now we have filtered and sorted by time data, we can split test data out
grouped_sorted = df.groupby(USER_COLUMN, group_keys=False)
test_data = grouped_sorted.tail(1).sort_values(by=USER_COLUMN)
# need to pop for each group
train_data = grouped_sorted.apply(lambda x: x.iloc[:-1])

In [9]:
train_data['target']=1
test_data['target']=1
train_data.head()

,userId,movieId,target
20,0,20,1
19,0,19,1
86,0,86,1
61,0,61,1
23,0,23,1


Since the movie lens data contains only positive examples, let us first define an utility function to generate negative samples.

In [10]:
class _TestNegSampler:
    def __init__(self, train_ratings, nb_users, nb_items, nb_neg):
        self.nb_neg = nb_neg
        self.nb_users = nb_users 
        self.nb_items = nb_items 

        # compute unique ids for quickly created hash set and fast lookup
        ids = (train_ratings[:, 0] * self.nb_items) + train_ratings[:, 1]
        self.set = set(ids)

    def generate(self, batch_size=128*1024):
        users = torch.arange(0, self.nb_users).reshape([1, -1]).repeat([self.nb_neg, 1]).transpose(0, 1).reshape(-1)

        items = [-1] * len(users)

        random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
        print('Generating validation negatives...')
        for idx, u in enumerate(tqdm.tqdm(users.tolist())):
            if not random_items:
                random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
            j = random_items.pop()
            while u * self.nb_items + j in self.set:
                if not random_items:
                    random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
                j = random_items.pop()

            items[idx] = j
        items = torch.LongTensor(items)
        return items

Next, we generate the negative samples for training.

In [11]:
sampler = _TestNegSampler(df.values, nb_users, nb_items, 500)  # using 500 negative samples
train_negs = sampler.generate()
train_negs = train_negs.reshape(-1, 500)

sampler = _TestNegSampler(df.values, nb_users, nb_items, 100)  # using 100 negative samples
test_negs = sampler.generate()
test_negs = test_negs.reshape(-1, 100)

Generating validation negatives...


100%|██████████| 69246500/69246500 [01:05<00:00, 1059133.29it/s]

Generating validation negatives...


100%|██████████| 13849300/13849300 [00:13<00:00, 1048928.58it/s]


In [12]:
import numpy as np

# generating negative samples for training
train_data_neg = np.zeros((train_negs.shape[0]*train_negs.shape[1],3), dtype=int)
idx = 0
for i in tqdm.tqdm(range(train_negs.shape[0])):
    for j in range(train_negs.shape[1]):
        train_data_neg[idx, 0] = i # user ID
        train_data_neg[idx, 1] = train_negs[i, j] # negative item ID
        idx += 1
    
# generating negative samples for testing
test_data_neg = np.zeros((test_negs.shape[0]*test_negs.shape[1],3), dtype=int)
idx = 0
for i in tqdm.tqdm(range(test_negs.shape[0])):
    for j in range(test_negs.shape[1]):
        test_data_neg[idx, 0] = i
        test_data_neg[idx, 1] = test_negs[i, j]
        idx += 1

100%|██████████| 138493/138493 [01:24<00:00, 1635.21it/s]


In [13]:
train_data_np= np.concatenate([train_data_neg, train_data.values])
np.random.shuffle(train_data_np)

test_data_np= np.concatenate([test_data_neg, test_data.values])
np.random.shuffle(test_data_np)


In [14]:
# HugeCTR expect user ID and item ID to be different, so we use 0 -> nb_users for user IDs and
# nb_users -> nb_users+nb_items for item IDs.
train_data_np[:,1] += nb_users 
test_data_np[:,1] += nb_users 

In [15]:
np.max(train_data_np[:,1])

165236

### Write HugeCTR data files

Next, we will write the data to disk using HugeCTR [Norm](../docs/configuration_file_setup.md#norm) dataset format.

In [16]:
from ctypes import c_longlong as ll
from ctypes import c_uint
from ctypes import c_float
from ctypes import c_int

def write_hugeCTR_data(huge_ctr_data, filename='huge_ctr_data.dat'):
    print("Writing %d samples"%huge_ctr_data.shape[0])
    with open(filename, 'wb') as f:
        #write header
        f.write(ll(0)) # 0: no error check; 1: check_num
        f.write(ll(huge_ctr_data.shape[0])) # the number of samples in this data file
        f.write(ll(1)) # dimension of label
        f.write(ll(1)) # dimension of dense feature
        f.write(ll(2)) # long long slot_num
        for _ in range(3): f.write(ll(0)) # reserved for future use

        for i in tqdm.tqdm(range(huge_ctr_data.shape[0])):
            f.write(c_float(huge_ctr_data[i,2])) # float label[label_dim];
            f.write(c_float(0)) # dummy dense feature
            f.write(c_int(1)) # slot 1 nnz: user ID
            f.write(c_uint(huge_ctr_data[i,0]))
            f.write(c_int(1)) # slot 2 nnz: item ID
            f.write(c_uint(huge_ctr_data[i,1]))

#### Train data

In [17]:
def generate_filelist(filelist_name, num_files, filename_prefix):
    with open(filelist_name, 'wt') as f:
        f.write('{0}\n'.format(num_files));
        for i in range(num_files):
            f.write('{0}_{1}.dat\n'.format(filename_prefix, i))

In [18]:
!rm -rf ./data/hugeCTR
!mkdir ./data/hugeCTR

for i, data_arr in enumerate(np.array_split(train_data_np,10)):
    write_hugeCTR_data(data_arr, filename='./data/hugeCTR/train_huge_ctr_data_%d.dat'%i)

generate_filelist('./data/hugeCTR/train_filelist.txt', 10, './data/hugeCTR/train_huge_ctr_data')

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:35<00:00, 249365.79it/s]

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:35<00:00, 250680.48it/s]

Writing 8910827 samples


...

100%|██████████| 8910827/8910827 [00:36<00:00, 247415.09it/s]

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:35<00:00, 247776.47it/s]

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:36<00:00, 247428.62it/s]


#### Test data

In [19]:
for i, data_arr in enumerate(np.array_split(test_data_np,10)):
    write_hugeCTR_data(data_arr, filename='./data/hugeCTR/test_huge_ctr_data_%d.dat'%i)
    
generate_filelist('./data/hugeCTR/test_filelist.txt', 10, './data/hugeCTR/test_huge_ctr_data')

Writing 1398780 samples


100%|██████████| 1398780/1398780 [00:05<00:00, 241195.10it/s]

Writing 1398780 samples


100%|██████████| 1398780/1398780 [00:05<00:00, 245859.52it/s]

Writing 1398780 samples


...

100%|██████████| 1398779/1398779 [00:05<00:00, 249214.55it/s]

Writing 1398779 samples


100%|██████████| 1398779/1398779 [00:05<00:00, 250584.19it/s]

Writing 1398779 samples


100%|██████████| 1398779/1398779 [00:05<00:00, 249278.58it/s]

<a id="3"></a>
## 3. HugeCTR DLRM training

In this section, we will train a DLRM network on the augmented movie lens data. First, we write the model config file.

In [20]:
%%writefile dlrm_config.json
{
  "solver": {
    "lr_policy": "fixed",
    "display": 1000,
    "max_iter":50000,
    "gpu": [0],
    "batchsize": 65536,
    "snapshot": 3000,
    "snapshot_prefix": "./hugeCTR_saved_model_DLRM/",
    "eval_interval": 3000,
    "max_eval_batches": 1000,
    "mixed_precision": 1024,
    "eval_metrics": ["AUC:1.0"]
  },
 
  "optimizer": {
    "type": "SGD",
    "update_type": "Local",
    "sgd_hparam": {
    "learning_rate": 0.1,
    "warmup_steps": 1000,
    "decay_start": 10000,
    "decay_steps": 40000,
    "end_lr": 1e-5
    }
  },


  "layers": [ 
      {
      "name": "data",
      "type": "Data",
      "slot_size_array": [138493 , 26744],
      "slot_size_array_orig": [138493 , 26744],
      "source": "./data/hugeCTR/train_filelist.txt",
      "eval_source": "./data/hugeCTR/test_filelist.txt",
      "check": "None",
      "cache_eval_data": true,
      "label": {
              "top": "label",
              "label_dim": 1
      },
      "dense": {
              "top": "dense",
              "dense_dim": 1
      },
      "sparse": [
              {
          "top": "data1",
          "type": "LocalizedSlot",
          "max_feature_num_per_sample": 2,
          "max_nnz": 1,
          "slot_num": 2
              }
      ]
    },

    {
      "name": "sparse_embedding1",
      "type": "LocalizedSlotSparseEmbeddingHash",
      "bottom": "data1",
      "top": "sparse_embedding1",
      "sparse_embedding_hparam": {
        "slot_size_array": [138493 , 26744],
        "embedding_vec_size": 64,
        "combiner": 0
      }
    },

    {
      "name": "fc1",
      "type": "FusedInnerProduct",
      "bottom": "dense",
      "top": "fc1",
      "fc_param": {
        "num_output": 64
      }
    },

    {
      "name": "fc2",
      "type": "FusedInnerProduct",
      "bottom": "fc1",
      "top": "fc2",
      "fc_param": {
        "num_output": 128
      }
    },

   
    {
      "name": "fc3",
      "type": "FusedInnerProduct",
      "bottom": "fc2",
      "top": "fc3",
      "fc_param": {
        "num_output": 64
      }
    },

    {
      "name": "interaction1",
      "type": "Interaction",
      "bottom": ["fc3", "sparse_embedding1"],
      "top": "interaction1"
    },

    {
      "name": "fc4",
      "type": "FusedInnerProduct",
      "bottom": "interaction1",
      "top": "fc4",
       "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc5",
      "type": "FusedInnerProduct",
      "bottom": "fc4",
      "top": "fc5",
      "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc6",
      "type": "FusedInnerProduct",
      "bottom": "fc5",
      "top": "fc6",
      "fc_param": {
        "num_output": 512
      }
    },

    {
      "name": "fc7",
      "type": "FusedInnerProduct",
      "bottom": "fc6",
      "top": "fc7",
      "fc_param": {
        "num_output": 256
      }
    },

    {
      "name": "fc8",
      "type": "InnerProduct",
      "bottom": "fc7",
      "top": "fc8",
      "fc_param": {
        "num_output": 1
      }
    },
    
    {
      "name": "loss",
      "type": "BinaryCrossEntropyLoss",
      "bottom": ["fc8","label"],
      "top": "loss"
    } 
  ]
}

Writing dlrm_config.json


In [21]:
!rm -rf ./hugeCTR_saved_model_DLRM/
!mkdir ./hugeCTR_saved_model_DLRM/

In [22]:
!CUDA_VISIBLE_DEVICES=0 huge_ctr --train ./dlrm_config.json

[0.001, init_start, ]
HugeCTR Version: 3.0.0
Config file: ./dlrm_config.json
[11d01h02m12s][HUGECTR][INFO]: batchsize_eval is not specified using default: 65536
[11d01h02m12s][HUGECTR][INFO]: Mixed Precision training with scaler: 1024 is enabled.
[11d01h02m12s][HUGECTR][INFO]: enable_tf32_compute is not specified using default: 0
[11d01h02m12s][HUGECTR][INFO]: TF32 Compute: OFF
[11d01h02m12s][HUGECTR][INFO]: algorithm_search is not specified using default: 1
[11d01h02m12s][HUGECTR][INFO]: Algorithm search: ON
[11d01h02m12s][HUGECTR][INFO]: cuda_graph is not specified using default: 1
[11d01h02m12s][HUGECTR][INFO]: CUDA Graph: ON
[11d01h02m12s][HUGECTR][INFO]: Export prediction: OFF
[11d01h02m12s][HUGECTR][INFO]: Global seed is 2762172846
[11d01h02m14s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-16GB
[11d01h02m14s][HUGECTR][INFO]: num_workers is not specified using default: 12
[11d01h02m14s][HUGECTR][INFO]: num of DataReader workers: 12
[11d01

[11d01h10m20s][HUGECTR][INFO]: Done
[488414, eval_start, 0.54, ]
[11d01h10m26s][HUGECTR][INFO]: Evaluation, AUC: 0.943296
[493965, eval_accuracy, 0.943296, 0.54, 27000, ]
[11d01h10m26s][HUGECTR][INFO]: Eval Time for 1000 iters: 5.550758s
[493965, eval_stop, 0.54, ]
[11d01h10m41s][HUGECTR][INFO]: Iter: 28000 Time(1000 iters): 20.169469s Loss: 0.149855 lr:0.030247
[11d01h10m55s][HUGECTR][INFO]: Iter: 29000 Time(1000 iters): 14.500178s Loss: 0.150364 lr:0.027560
[11d01h11m10s][HUGECTR][INFO]: Iter: 30000 Time(1000 iters): 14.501092s Loss: 0.155115 lr:0.024998
[11d01h11m10s][HUGECTR][INFO]: Rank0: Dump hash table from GPU0
[11d01h11m10s][HUGECTR][INFO]: Rank0: Write hash table <key,slot_id,value> pairs to file
[11d01h11m10s][HUGECTR][INFO]: Done
[537580, eval_start, 0.6, ]
[11d01h11m15s][HUGECTR][INFO]: Evaluation, AUC: 0.943183
[543136, eval_accuracy, 0.943183, 0.6, 30000, ]
[11d01h11m15s][HUGECTR][INFO]: Eval Time for 1000 iters: 5.555897s
[543136, eval_stop, 0.6, ]
[11d01h11m30s][HUGECT

<a id="4"></a>
## 4. Answer item similarity with DLRM embedding

In this section, we demonstrate how the output of HugeCTR training can be used to carry out simple inference tasks. Specifically, we will show that the movie embeddings can be used for simple item-to-item similarity queries. Such a simple inference can be used as an efficient candidate generator to generate a small set of cadidates prior to deep learning model re-ranking. 

First, we read the embedding tables and extract the movie embeddings.

In [23]:
import struct 
import pickle
import numpy as np

key_type = 'I32' # {'I64', 'I32'}, default is 'I32'
key_type_map = {"I32": ["I", 4], "I64": ["q", 8]}

embedding_vec_size = 64

HUGE_CTR_VERSION = 2.21 # set HugeCTR version here, 2.2 for v2.2, 2.21 for v2.21

if HUGE_CTR_VERSION <= 2.2:
    each_key_size = key_type_map[key_type][1] + key_type_map[key_type][1] + 4 * embedding_vec_size
else:
    each_key_size = key_type_map[key_type][1] + 8 + 4 * embedding_vec_size

In [24]:
embedding_table = [{},{}]

with open('./hugeCTR_saved_model_DLRM/0_sparse_9000.model', 'rb') as file:
    try:
        while True:
            buffer = file.read(each_key_size)
            if len(buffer) == 0:
                break
            if HUGE_CTR_VERSION <= 2.2:
                key, slot_id = struct.unpack("2" + key_type_map[key_type][0], 
                                             buffer[0: 2*key_type_map[key_type][1]])
                values = struct.unpack(str(embedding_vec_size) + "f", buffer[2*key_type_map[key_type][1]: ])
            else:
                key = struct.unpack(key_type_map[key_type][0], buffer[0 : key_type_map[key_type][1]])[0]
                slot_id = struct.unpack("Q", buffer[key_type_map[key_type][1] : key_type_map[key_type][1] + 8])[0]
                values = struct.unpack(str(embedding_vec_size) + "f", buffer[key_type_map[key_type][1] + 8: ])
            if slot_id==0:
                embedding_table[slot_id][key] = values
            elif slot_id==1:
                embedding_table[slot_id][key - 138493] = values 
            else:
                raise(Exception("Slot ID not found - %d"%slot_id))
            
    except BaseException as error:
        print(error)

In [25]:
item_embedding = np.zeros((26744, embedding_vec_size), dtype='float')
for i in range(len(embedding_table[1])):
    item_embedding[i] = embedding_table[1][i]
    

In [26]:
len(embedding_table[1])

26744

### Answer nearest neighbor queries


In [27]:
from scipy.spatial.distance import cdist

def find_similar_movies(nn_movie_id, item_embedding, k=10, metric="euclidean"):
    #find the top K similar items according to one of the distance metric: cosine or euclidean
    sim = 1-cdist(item_embedding, item_embedding[nn_movie_id].reshape(1, -1), metric=metric)
   
    return sim.squeeze().argsort()[-k:][::-1]

In [28]:
with open('./mappings.pickle', 'rb') as handle:
    movies_mapping = pickle.load(handle)["items"]

nn_to_movies = movies_mapping
movies_to_nn = {}
for i in range(len(movies_mapping)):
    movies_to_nn[movies_mapping[i]] = i

import pandas as pd
movies = pd.read_csv("./data/ml-20m/movies.csv", index_col="movieId")


In [29]:
for movie_ID in range(1,1000):
    try:
        print("Query: ", movies.loc[movie_ID]["title"], movies.loc[movie_ID]["genres"])

        print("Similar movies: ")
        similar_movies = find_similar_movies(movies_to_nn[movie_ID], item_embedding)

        for i in similar_movies:
            print(nn_to_movies[i], movies.loc[nn_to_movies[i]]["title"], movies.loc[nn_to_movies[i]]["genres"])
        print("=================================\n")
    except Exception as e:
        pass

Query:  Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
Similar movies: 
1 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
1210 Star Wars: Episode VI - Return of the Jedi (1983) Action|Adventure|Sci-Fi
150 Apollo 13 (1995) Adventure|Drama|IMAX
457 Fugitive, The (1993) Thriller
2571 Matrix, The (1999) Action|Sci-Fi|Thriller
592 Batman (1989) Action|Crime|Thriller
32 Twelve Monkeys (a.k.a. 12 Monkeys) (1995) Mystery|Sci-Fi|Thriller
527 Schindler's List (1993) Drama|War
589 Terminator 2: Judgment Day (1991) Action|Sci-Fi
1196 Star Wars: Episode V - The Empire Strikes Back (1980) Action|Adventure|Sci-Fi

Query:  Jumanji (1995) Adventure|Children|Fantasy
Similar movies: 
2 Jumanji (1995) Adventure|Children|Fantasy
25 Leaving Las Vegas (1995) Drama|Romance
4886 Monsters, Inc. (2001) Adventure|Animation|Children|Comedy|Fantasy
1307 When Harry Met Sally... (1989) Comedy|Romance
4995 Beautiful Mind, A (2001) Drama|Romance
919 Wizard of Oz, The (1939) Adventure|Children

Query:  Assassins (1995) Action|Crime|Thriller
Similar movies: 
23 Assassins (1995) Action|Crime|Thriller
94 Beautiful Girls (1996) Comedy|Drama|Romance
20 Money Train (1995) Action|Comedy|Crime|Drama|Thriller
1958 Terms of Endearment (1983) Comedy|Drama
2841 Stir of Echoes (1999) Horror|Mystery|Thriller
3681 For a Few Dollars More (Per qualche dollaro in più) (1965) Action|Drama|Thriller|Western
3638 Moonraker (1979) Action|Adventure|Sci-Fi|Thriller
147 Basketball Diaries, The (1995) Drama
1588 George of the Jungle (1997) Children|Comedy
99114 Django Unchained (2012) Action|Drama|Western

Query:  Powder (1995) Drama|Sci-Fi
Similar movies: 
24 Powder (1995) Drama|Sci-Fi
3082 World Is Not Enough, The (1999) Action|Adventure|Thriller
441 Dazed and Confused (1993) Comedy
1223 Grand Day Out with Wallace and Gromit, A (1989) Adventure|Animation|Children|Comedy|Sci-Fi
7254 The Butterfly Effect (2004) Drama|Sci-Fi|Thriller
2580 Go (1999) Comedy|Crime
3039 Trading Places (1983) Comedy
531 Secr

46 How to Make an American Quilt (1995) Drama|Romance
1810 Primary Colors (1998) Comedy|Drama
2950 Blue Lagoon, The (1980) Adventure|Drama|Romance
2143 Legend (1985) Adventure|Fantasy|Romance
4623 Major League (1989) Comedy
234 Exit to Eden (1994) Comedy
6287 Anger Management (2003) Comedy
5171 Time Machine, The (2002) Action|Adventure|Sci-Fi
6936 Elf (2003) Children|Comedy|Fantasy
951 His Girl Friday (1940) Comedy|Romance

Query:  Seven (a.k.a. Se7en) (1995) Mystery|Thriller
Similar movies: 
47 Seven (a.k.a. Se7en) (1995) Mystery|Thriller
377 Speed (1994) Action|Romance|Thriller
1198 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981) Action|Adventure
1270 Back to the Future (1985) Adventure|Comedy|Sci-Fi
1196 Star Wars: Episode V - The Empire Strikes Back (1980) Action|Adventure|Sci-Fi
364 Lion King, The (1994) Adventure|Animation|Children|Drama|Musical|IMAX
50 Usual Suspects, The (1995) Crime|Mystery|Thriller
590 Dances with Wolves (1990) Adventure|Drama|W

69 Friday (1995) Comedy
1955 Kramer vs. Kramer (1979) Drama
54997 3:10 to Yuma (2007) Action|Crime|Drama|Western
2402 Rambo: First Blood Part II (1985) Action|Adventure|Thriller
5219 Resident Evil (2002) Action|Horror|Sci-Fi|Thriller
3095 Grapes of Wrath, The (1940) Drama
1416 Evita (1996) Drama|Musical
6534 Hulk (2003) Action|Adventure|Sci-Fi
3986 6th Day, The (2000) Action|Sci-Fi|Thriller
3087 Scrooged (1988) Comedy|Fantasy|Romance

Query:  From Dusk Till Dawn (1996) Action|Comedy|Horror|Thriller
Similar movies: 
70 From Dusk Till Dawn (1996) Action|Comedy|Horror|Thriller
2572 10 Things I Hate About You (1999) Comedy|Romance
342 Muriel's Wedding (1994) Comedy
1997 Exorcist, The (1973) Horror|Mystery
59315 Iron Man (2008) Action|Adventure|Sci-Fi
2890 Three Kings (1999) Action|Adventure|Comedy|Drama|War
158 Casper (1995) Adventure|Children
333 Tommy Boy (1995) Comedy
1276 Cool Hand Luke (1967) Drama
2019 Seven Samurai (Shichinin no samurai) (1954) Action|Adventure|Drama

Query:  Fair G

Query:  Vampire in Brooklyn (1995) Comedy|Horror|Romance
Similar movies: 
93 Vampire in Brooklyn (1995) Comedy|Horror|Romance
3685 Prizzi's Honor (1985) Comedy|Drama|Romance
7669 Pride and Prejudice (1995) Drama|Romance
3741 Badlands (1973) Crime|Drama|Thriller
7569 You Only Live Twice (1967) Action|Adventure|Sci-Fi|Thriller
91542 Sherlock Holmes: A Game of Shadows (2011) Action|Adventure|Comedy|Crime|Mystery|Thriller
3836 Kelly's Heroes (1970) Action|Comedy|War
437 Cops and Robbersons (1994) Comedy
3040 Meatballs (1979) Comedy
2322 Soldier (1998) Action|Sci-Fi|War

Query:  Beautiful Girls (1996) Comedy|Drama|Romance
Similar movies: 
94 Beautiful Girls (1996) Comedy|Drama|Romance
23 Assassins (1995) Action|Crime|Thriller
54997 3:10 to Yuma (2007) Action|Crime|Drama|Western
1958 Terms of Endearment (1983) Comedy|Drama
99114 Django Unchained (2012) Action|Drama|Western
514 Ref, The (1994) Comedy
1619 Seven Years in Tibet (1997) Adventure|Drama|War
1243 Rosencrantz and Guildenstern Are De

...

902 Breakfast at Tiffany's (1961) Drama|Romance
5995 Pianist, The (2002) Drama|War
762 Striptease (1996) Comedy|Crime
6942 Love Actually (2003) Comedy|Drama|Romance
1047 Long Kiss Goodnight, The (1996) Action|Drama|Thriller
1339 Dracula (Bram Stoker's Dracula) (1992) Fantasy|Horror|Romance|Thriller
1293 Gandhi (1982) Drama
5377 About a Boy (2002) Comedy|Drama|Romance
1343 Cape Fear (1991) Thriller
7254 The Butterfly Effect (2004) Drama|Sci-Fi|Thriller

Query:  Vertigo (1958) Drama|Mystery|Romance|Thriller
Similar movies: 
903 Vertigo (1958) Drama|Mystery|Romance|Thriller
1777 Wedding Singer, The (1998) Comedy|Romance
172 Johnny Mnemonic (1995) Action|Sci-Fi|Thriller
953 It's a Wonderful Life (1946) Drama|Fantasy|Romance
163 Desperado (1995) Action|Romance|Western
1676 Starship Troopers (1997) Action|Sci-Fi
1954 Rocky (1976) Drama
168 First Knight (1995) Action|Drama|Romance
1500 Grosse Pointe Blank (1997) Comedy|Crime|Romance
7147 Big Fish (2003) Drama|Fantasy|Romance

Query:  Rear Win

925 Golden Earrings (1947) Adventure|Romance|War
95856 Knick Knack (1989) Animation|Children
4781 Megiddo: The Omega Code 2 (2001) Action|Adventure|Fantasy|Sci-Fi|Thriller
3805 Knightriders (1981) Action|Adventure|Drama
3960 Haunted (1995) Drama|Thriller
7191 Blame It on the Bellboy (1992) Comedy
1174 Dead Tired (Grosse Fatigue) (1994) Comedy
5930 Brimstone and Treacle (1982) Drama|Thriller
92048 Whistleblower, The (2010) Drama|Thriller
49688 Dam Busters, The (1955) Action|Drama|War

Query:  All About Eve (1950) Drama
Similar movies: 
926 All About Eve (1950) Drama
3984 Diamonds Are Forever (1971) Action|Adventure|Thriller
1956 Ordinary People (1980) Drama
8529 Terminal, The (2004) Comedy|Drama|Romance
2053 Honey, I Blew Up the Kid (1992) Children|Comedy|Sci-Fi
1687 Jackal, The (1997) Action|Thriller
9 Sudden Death (1995) Action
8983 House of Flying Daggers (Shi mian mai fu) (2004) Action|Drama|Romance
2808 Universal Soldier (1992) Action|Sci-Fi
4007 Wall Street (1987) Drama

Query:  W

948 Giant (1956) Drama|Romance|Western
1111 Microcosmos (Microcosmos: Le peuple de l'herbe) (1996) Documentary
3707 9 1/2 Weeks (Nine 1/2 Weeks) (1986) Drama|Romance
2439 Affliction (1997) Drama
569 Little Big League (1994) Comedy|Drama
8907 Shark Tale (2004) Animation|Children|Comedy
3649 American Gigolo (1980) Drama
4902 Devil's Backbone, The (Espinazo del diablo, El) (2001) Drama|Fantasy|Horror|Thriller|War
6294 Bulletproof Monk (2003) Action|Adventure|Sci-Fi
3163 Topsy-Turvy (1999) Comedy|Drama|Musical

Query:  East of Eden (1955) Drama
Similar movies: 
949 East of Eden (1955) Drama
2901 Phantasm (1979) Horror|Sci-Fi
437 Cops and Robbersons (1994) Comedy
1255 Bad Taste (1987) Comedy|Horror|Sci-Fi
56949 27 Dresses (2008) Comedy|Romance
80489 Town, The (2010) Crime|Drama|Thriller
3007 American Movie (1999) Documentary
259 Kiss of Death (1995) Crime|Drama|Thriller
1889 Insomnia (1997) Drama|Mystery|Thriller
340 War, The (1994) Adventure|Drama|War

Query:  Thin Man, The (1934) Comedy|C

971 Cat on a Hot Tin Roof (1958) Drama
1888 Hope Floats (1998) Comedy|Drama|Romance
1616 Peacemaker, The (1997) Action|Thriller|War
2764 Thomas Crown Affair, The (1968) Crime|Drama|Romance|Thriller
5620 Sweet Home Alabama (2002) Comedy|Romance
4857 Fiddler on the Roof (1971) Drama|Musical
1643 Mrs. Brown (a.k.a. Her Majesty, Mrs. Brown) (1997) Drama|Romance
1126 Drop Dead Fred (1991) Comedy|Fantasy
47099 Pursuit of Happyness, The (2006) Drama
63113 Quantum of Solace (2008) Action|Adventure|Thriller

Query:  Last Time I Saw Paris, The (1954) Drama
Similar movies: 
972 Last Time I Saw Paris, The (1954) Drama
6895 Normal (2003) Drama
101997 Scary Movie 5 (Scary MoVie) (2013) Comedy
3313 Class Reunion (1982) Comedy
128 Jupiter's Wife (1994) Documentary
6010 Double Whammy (2001) Action|Comedy|Drama
99574 Promised Land (2012) Drama
5186 Honeysuckle Rose (a.k.a. On the Road Again) (1980) Drama|Romance
3940 Slumber Party Massacre III (1990) Horror
55156 Unreasonable Man, An (2006) Documentary


Similar movies: 
996 Last Man Standing (1996) Action|Crime|Drama|Thriller
5878 Talk to Her (Hable con Ella) (2002) Drama|Romance
1173 Cook the Thief His Wife & Her Lover, The (1989) Comedy|Drama
1126 Drop Dead Fred (1991) Comedy|Fantasy
6659 Tremors (1990) Comedy|Horror|Sci-Fi
2379 Police Academy 2: Their First Assignment (1985) Comedy|Crime
6796 Boyz N the Hood (1991) Crime|Drama
1078 Bananas (1971) Comedy|War
3825 Coyote Ugly (2000) Comedy|Drama|Romance
3044 Dead Again (1991) Mystery|Romance|Thriller

Query:  Caught (1996) Drama|Thriller
Similar movies: 
997 Caught (1996) Drama|Thriller
3211 Cry in the Dark, A (1988) Drama
72294 Christmas Carol, A (2009) Animation|Children|Drama|Fantasy|IMAX
8772 Spy Who Came in from the Cold, The (1965) Drama|Thriller
33681 Adventures of Sharkboy and Lavagirl 3-D, The (2005) Action|Adventure|Children|Fantasy
1583 Simple Wish, A (1997) Children|Fantasy
44849 Renaissance (2006) Action|Animation|Film-Noir|Sci-Fi|Thriller
41527 Paradise Now (2005) Crime